

## 一、训练

### 1.1 导入mda数据

读取2001-2022年的**管理层讨论与分析mda**数据

In [ ]:
import pandas as pd

df = pd.read_excel('mda01-22.xlsx')
df.head()

<br>
### 1.2 构造语料

In [ ]:
with open('mda01-22.txt', 'a+', encoding='utf-8') as f:
    text = ''.join(df['text'])
    f.write(text)

<br>

### 1.3 配置cntext环境

使用2.0.0版本cntext库(该版本暂不开源，需付费购买)。 将得到的 **cntext-2.0.0-py3-none-any.whl** 文件放置于电脑桌面，  win系统打开**cmd**(Mac打开terminal)， 输入如下命令(将工作环境切换至桌面)

```
cd desktop
```
<br>


个别Win用户如无效，试试``cd Desktop`` 。



继续在cmd (terminal) 中执行如下命令安装cntext2.0.0

```
pip3 install cntext-2.0.0-py3-none-any.whl 
```

<br>



<br>

### 1.4 训练word2vec

设置模型参数配置

- mda01-22 使用2001-2022年度mda数据训练
- 200 嵌入的维度数，即每个词的向量长度是200
- 6 词语上下文的窗口是6


In [ ]:

%%time  #程序结束后，可查看总的运行时间
import cntext as ct

w2v = ct.W2VModel(corpus_file='mda01-22.txt')
w2v.train(vector_size=200, window_size=6, min_count=6, save_dir='Word2Vec')


经过不到两个小时时间， 训练出的中国A股市场词向量模型，词汇量 914058， 模型文件 1.49G。模型可广泛用于经济管理等领域概念(情感)词典的构建或扩展。 

- **mda01-22.200.6.bin**
- **mda01-22.200.6.bin.syn1neg.npy**
- **mda01-22.200.6.bin.wv.vectors.npy**


## 二、导入模型
需要用到两个自定义函数load_w2v、expand_dictionary，源代码太长，为了提高阅读体验， 放在文末。大家记得用这两个函数前一定要先导入

In [8]:
#先导入load_w2v、expand_dictionary函数源代码


#读取模型文件
w2v_model = load_w2v(w2v_path='mda01-22.200.6.bin')
w2v_model

Loading word2vec model...


### 三、w2v的使用
- 查看词汇量
- 查询某词向量
- 查看多个词的均值向量

更多内容，建议查看下gensim库的文档

In [9]:
#词汇量
len(w2v_model.wv.index_to_key)

914058

In [10]:
#查询某词的词向量
w2v_model.wv.get_vector('创新')

array([-1.36441350e-01, -2.02002168e+00, -1.49168205e+00,  2.65202689e+00,
        1.49721682e+00,  2.14851022e+00, -1.54925853e-01, -2.25241160e+00,
       -3.58773202e-01,  1.54530525e+00, -7.62950361e-01, -9.77181852e-01,
        6.70365512e-01, -3.20203233e+00,  3.18079638e+00,  1.66510820e+00,
        9.80131567e-01,  1.62199986e+00,  1.80585206e+00,  4.08179426e+00,
       -1.26518166e+00,  3.75929743e-01,  5.72038591e-01,  1.16134119e+00,
        2.55617023e+00, -2.25110960e+00, -2.61538339e+00, -5.71992218e-01,
        8.70356798e-01, -1.85045290e+00, -2.85597444e-01, -9.15628672e-01,
       -2.03667688e+00,  2.11716801e-01,  2.94088912e+00, -2.32688546e+00,
        2.20858502e+00,  8.81347775e-01, -7.99135566e-01, -8.61206651e-01,
       -4.45446587e+00, -1.73757005e+00, -3.36678886e+00, -2.82611530e-02,
       -1.62726247e+00, -8.49750221e-01,  4.13731128e-01, -1.62519825e+00,
        3.03865957e+00, -1.39746085e-01,  8.22233260e-01, -7.97697455e-02,
        1.72468078e+00,  

In [11]:
#查询多个词的词向量
w2v_model.wv.get_mean_vector(['创新', '研发'])

array([ 0.03019853, -0.01928307, -0.05371316,  0.00053774,  0.02516318,
        0.10103251, -0.03914721, -0.08307559,  0.00444389,  0.09456791,
       -0.05761364, -0.03459097,  0.04394419, -0.10181106,  0.1418381 ,
        0.05334964,  0.01820264,  0.01493831,  0.01626587,  0.17402864,
       -0.02859601,  0.04538149,  0.03768233,  0.05431981,  0.15405464,
       -0.03632693, -0.08566202, -0.00595666,  0.08378439, -0.11071078,
       -0.05904576, -0.06451955, -0.1076955 ,  0.05141645,  0.11710279,
       -0.09403889,  0.08633652, -0.06743232,  0.00328483,  0.01589498,
       -0.11226317, -0.05367877, -0.057222  , -0.00685401, -0.04531868,
       -0.02090884,  0.01426806, -0.04787309,  0.1325518 , -0.00498158,
        0.01912023, -0.02292867,  0.08855374,  0.07697155,  0.01407153,
       -0.02378988,  0.03745927,  0.00889686,  0.12555045,  0.04007044,
        0.06247196,  0.04912657, -0.06158784,  0.06346396,  0.00197599,
       -0.04995281,  0.05125345, -0.01584197,  0.07572784,  0.02

有了每个词或者概念的向量，可以结合cntext旧版本单语言模型内的态度偏见的度量。

<br>

## 四、扩展词典
做词典法的文本分析，最重要的是有自己的领域词典。之前受限于技术难度，文科生的我也一直在用形容词的通用情感词典。现在依托word2vec技术， 可以加速人工构建的准确率和效率。


下面是在 mda01-22.200.6.bin 上做的词典扩展测试，函数expand_dictionary会根据种子词选取最准确的topn个词。

In [12]:
#短视主义词  实验
expand_dictionary(wv=w2v_model.wv, 
                  seedwords=['抓紧', '立刻', '月底', '年底', '年终', '争取', '力争'],
                  topn=30)

['抓紧',
 '立刻',
 '月底',
 '年底',
 '年终',
 '争取',
 '力争',
 '争取',
 '力争',
 '年底',
 '月底',
 '3月底',
 '尽快',
 '上半年',
 '努力争取',
 '年内实现',
 '抓紧',
 '工作争取',
 '尽早',
 '6月底',
 '工作力争',
 '7月份',
 '年底完成',
 '确保',
 '早日',
 '有望',
 '全力',
 '创造条件',
 '3月份',
 '加紧',
 '力争实现',
 '力争今年',
 '月底前',
 '10月底',
 '4月份',
 '继续',
 '月初']

In [14]:
expand_dictionary(wv=w2v_model.wv, 
                  seedwords=['团结', '拼搏',  '克服',  '勇攀高峰',  '友善',  '进取'],
                  topn=30)

['团结',
 '拼搏',
 '克服',
 '勇攀高峰',
 '友善',
 '进取',
 '拼搏',
 '艰苦奋斗',
 '团结拼搏',
 '勇于担当',
 '锐意进取',
 '勇气',
 '团结',
 '团结奋进',
 '团结一致',
 '顽强拼搏',
 '上下一心',
 '实干',
 '拼搏进取',
 '积极进取',
 '奋力拼搏',
 '奋进',
 '坚定信念',
 '团结一心',
 '精诚团结',
 '顽强',
 '踏实',
 '团结协作',
 '求真务实',
 '团结奋斗',
 '奋发有为',
 '同心协力',
 '脚踏实地',
 '开拓进取',
 '进取',
 '勇于']

In [15]:
expand_dictionary(wv=w2v_model.wv, 
                  seedwords=['创新', '科技',  '研发',  '技术',  '标准'],
                  topn=30)

['创新',
 '科技',
 '研发',
 '技术',
 '标准',
 '技术创新',
 '技术研发',
 '先进技术',
 '关键技术',
 '创新性',
 '前沿技术',
 '科技创新',
 '技术应用',
 '产品开发',
 '自主创新',
 '新技术',
 '科研',
 '产品研发',
 '自主研发',
 '技术开发',
 '工艺技术',
 '技术标准',
 '基础研究',
 '集成创新',
 '核心技术',
 '成熟技术',
 '研发创新',
 '理论技术',
 '前沿技术研发',
 '工艺',
 '科技成果',
 '技术研究',
 '标准制定',
 '技术装备',
 '技术相结合']

In [16]:
expand_dictionary(wv=w2v_model.wv, 
                  seedwords=['竞争', '竞争力'],
                  topn=30)

['竞争',
 '竞争力',
 '竞争能力',
 '市场竞争',
 '竞争优势',
 '市场竞争力',
 '竞',
 '竞争实力',
 '激烈竞争',
 '参与市场竞争',
 '国际竞争',
 '市场竞争能力',
 '竞争态势',
 '市场竞争优势',
 '行业竞争',
 '综合竞争力',
 '竞争对手',
 '未来市场竞争',
 '产品竞争力',
 '之间竞争',
 '核心竞争力',
 '参与竞争',
 '核心竞争能力',
 '竞争日趋激烈',
 '国际化竞争',
 '国际竞争力',
 '竟争力',
 '市场化竞争',
 '同质化竞争',
 '竞争力关键',
 '价格竞争',
 '整体竞争力']

In [17]:
expand_dictionary(wv=w2v_model.wv, 
                  seedwords=['疫情', '扩散', '防控', '反复', '冲击'],
                  topn=30)

['疫情',
 '扩散',
 '防控',
 '反复',
 '冲击',
 '蔓延',
 '疫情',
 '疫情爆发',
 '疫情冲击',
 '新冠疫情',
 '肆虐',
 '新冠肺炎',
 '疫情蔓延',
 '本次疫情',
 '散发',
 '疫情扩散',
 '疫情影响',
 '疫情反复',
 '疫情传播',
 '肺炎疫情',
 '国内疫情',
 '击',
 '各地疫情',
 '疫情全球',
 '疫情多点',
 '全球疫情',
 '持续蔓延',
 '多点散发',
 '疫情导致',
 '疫情暴发',
 '病毒疫情',
 '疫情持续',
 '疫情初期',
 '疫情出现',
 '防控措施']

In [18]:
expand_dictionary(wv=w2v_model.wv, 
                  seedwords=['旧', '老', '后', '落后'],
                  topn=30)

['旧',
 '老',
 '后',
 '落后',
 '老',
 '旧',
 '陈旧',
 '老旧',
 '淘汰',
 '低效率',
 '低效',
 '部分老旧',
 '进行改造',
 '老旧设备',
 '工艺落后',
 '设备陈旧',
 '能耗高',
 '更新改造',
 '落后工艺',
 '技术落后',
 '改造',
 '翻新',
 '简陋',
 '旧设备',
 '拆除',
 '现象严重',
 '原有',
 '相对落后',
 '产能淘汰',
 '加快淘汰',
 '搬',
 '替换',
 '大批',
 '迁']

In [1]:
from gensim.models import KeyedVectors
from pathlib import Path


def load_w2v(w2v_path):
    """
    Load word2vec model

    Args:
        w2v_path (str): path of word2vec model

    Returns:
        model: word2vec model
    """
    print('Loading word2vec model...')
    model = KeyedVectors.load(w2v_path)
    return model


def expand_dictionary(wv, seedwords, topn=100):
    """
    According to the seed word file, select the top n words with the most similar semantics and save them in the directory save_dir.
    
    Args:
        wv (Word2VecKeyedVectors): the word embedding model
        seedwords (list): 种子词
        topn (int, optional): Set the number of most similar words to retrieve to topn. Defaults to 100.
        save_dir (str, optional): the directory to save the candidate words. Defaults to 'Word2Vec'.
    
    Returns:
    """
    simidx_scores = []

    similars_candidate_idxs = [] #the candidate words of seedwords
    dictionary = wv.key_to_index
    seedidxs = [] #transform word to index
    for seed in seedwords:
        if seed in dictionary:
            seedidx = dictionary[seed]
            seedidxs.append(seedidx)
    for seedidx in seedidxs:
        # sims_words such as [('by', 0.99984), ('or', 0.99982), ('an', 0.99981), ('up', 0.99980)]
        sims_words = wv.similar_by_word(seedidx, topn=topn)
        #Convert words to index and store them
        similars_candidate_idxs.extend([dictionary[sim[0]] for sim in sims_words])
    similars_candidate_idxs = set(similars_candidate_idxs)
    
    for idx in similars_candidate_idxs:
        score = wv.n_similarity([idx], seedidxs)
        simidx_scores.append((idx, score))
    simidxs = [w[0] for w in sorted(simidx_scores, key=lambda k:k[1], reverse=True)]

    simwords = [str(wv.index_to_key[idx]) for idx in simidxs][:topn]

    resultwords = []
    resultwords.extend(seedwords)
    resultwords.extend(simwords)
    
    return resultwords